### This is for loading the filtered data in case the original file is either too large or can not be found

In [1]:
import pandas as pd
import numpy as np
import torch
import random
import ast
import json

In [2]:
df_final = pd.read_csv(filepath_or_buffer='./spotify_data.csv')
df_final.head()

,F,X,F_vector,session_id,Z
0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.248357,"[0.06952813690749203, -0.011178137654304811, 0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",1.930868,"[0.06952381845489436, -0.0111804461479361, 0.0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
2,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",3.323844,"[0.06952657157587336, -0.01118143003243, 0.019...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",4.761515,"[0.06952381845489436, -0.0111804461479361, 0.0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",4.882923,"[0.06946601984841431, -0.011168818418563512, 0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."


In [3]:
df_final['F'] = df_final['F'].apply(json.loads)
df_final['F_vector'] = df_final['F_vector'].apply(json.loads)
df_final['Z'] = df_final['Z'].apply(json.loads)

df_final.head()

,F,X,F_vector,session_id,Z
0,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]",1.248357,"[0.06952813690749203, -0.011178137654304811, 0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
1,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",1.930868,"[0.06952381845489436, -0.0111804461479361, 0.0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
2,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",3.323844,"[0.06952657157587336, -0.01118143003243, 0.019...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
3,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]",4.761515,"[0.06952381845489436, -0.0111804461479361, 0.0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."
4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]",4.882923,"[0.06946601984841431, -0.011168818418563512, 0...",65_0008dac3-8379-4369-90a1-b7ceccbb2d51,"[0.3865182998978819, -0.04937550520932488, 0.0..."


In [4]:
df_final['F'] = df_final['F'].apply(np.array)
df_final['F_vector'] = df_final['F_vector'].apply(np.array)
df_final['Z'] = df_final['Z'].apply(np.array)

In [5]:
N = df_final['session_id'].nunique()
T = df_final.groupby('session_id').size().iloc[0]
l = len(df_final['F'].iloc[0])
z_dim = len(df_final['Z'].iloc[0])

print(f'We have a time series of {N} X {T} X {l}. and z of dimension {z_dim}')

We have a time series of 6000 X 10 X 15. and z of dimension 5


In [6]:
F_array = np.zeros((N, T, l))
F_vector_array = np.zeros((N, T, l))
X_array = np.zeros((N, T))
Z_array = np.zeros((N, z_dim))


for i, (session_id, group) in enumerate(df_final.groupby('session_id')):
    # Ensure the group is sorted
    group = group.sort_index()
    F_array[i, :len(group), :] = np.array(group['F'].tolist())
    F_vector_array[i, :len(group), :] = np.array(group['F_vector'].tolist())
    X_array[i, :len(group)] = group['X'].values
    Z_array[i] = group['Z'].iloc[0]

# Z_array = session_embeddings[:, :z_dim]

In [7]:
print(f'Read X of shape {X_array.shape}, F of shape {F_array.shape}, F_vector of shape {F_vector_array.shape} and Z of shape {Z_array.shape}')

np.save('./data/x.npy', X_array)
np.save('./data/f_vec.npy', F_vector_array)
np.save('./data/f.npy', F_array)
np.save('./data/z.npy', Z_array)

Read X of shape (6000, 10), F of shape (6000, 10, 15), F_vector of shape (6000, 10, 15) and Z of shape (6000, 5)


In [8]:
F_array[0, :3]

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [9]:
F_vector_array[0, :3]

array([[ 0.06952814, -0.01117814,  0.01909687, -0.02114447,  0.032886  ,
        -0.00681013, -0.01538108,  0.02479554, -0.03485195, -0.03635439,
        -0.01490247,  0.04607297, -0.00842362,  0.00661841, -0.02079743],
       [ 0.06952382, -0.01118045,  0.01910232, -0.02114167,  0.03288622,
        -0.00680267, -0.01537809,  0.024803  , -0.03485931, -0.03635278,
        -0.01490233,  0.04606793, -0.00841987,  0.00661901, -0.02080396],
       [ 0.06952657, -0.01118143,  0.0190919 , -0.02113861,  0.03288055,
        -0.00681184, -0.01537577,  0.02477858, -0.0348463 , -0.03632605,
        -0.01490079,  0.04606742, -0.00842519,  0.00661654, -0.02078746]])

In [10]:
X_array[0, :3]

array([1.24835708, 1.93086785, 3.32384427])

In [11]:
Z_array[0]

array([ 0.3865183 , -0.04937551,  0.07676365, -0.08082001,  0.12558457])